# Calling Shodan via Rest-API

#### First setup API Key
 Shodan REST API https://developer.shodan.io/ provided with the proper API key

In [272]:
$APIKey = "xxxxxxxYMDeqVS3HxxxxxxOi18yxxxxx"  ## API Key obtained from Shodan
$credentialPath = ".\credentials\shodan.credential"                ## where to store credentials

#### Import Shodan PS Module

In [1]:
$host.UI.RawUI.BufferSize = [System.Management.Automation.Host.Size]::new(200, 50) ## Corrects the output
Import-Module ".\modules\shodan.psm1" -force

# Grabbing commands available
Get-Command -module "shodan" | Select-Object -Property Name | Format-Wide -Column 4
Out-Display -input "Type <font color=Yellow>'Get-Help &lt;command&gt; -Examples</font>' to get help on any modules"

The below script needs to be able to find the current output cell; this is an easy method to get it.


Get-ShodanAPIInfo                                 Get-ShodanDNSResolve                              Get-ShodanDNSReverse                              Get-ShodanHostService
Get-ShodanMyIP                                    Get-ShodanService                                 Measure-ShodanExploit                             Measure-ShodanHost
Read-ShodanAPIKey                                 Search-ShodanExploit                              Search-ShodanHost                                 Set-ShodanAPIKey



Type 'Get-Help <command> -Examples ' to get help on any modules

# first build credentials

In [ ]:
# Converts the API key to a SecureString
if (!$credentialPath) {$credentialPath =".\credentials\shodan.credential"}
if (!$APIKey) {$APIKey = Read-host "No APIKey provided, please enter Shodan API Key you got: "}
$SecureKeyString = ConvertTo-SecureString -String $APIKey -AsPlainText -Force

# Get a master password used to encrypt and decrypt Key
Write-Host -Foreground cyan "`nPlease enter a master password that will be used to encrypt/decrypt this API Key stored locally";
$MasterPassword = Read-Host -AsSecureString
$Credentials = New-Object System.Management.Automation.PSCredential -ArgumentList 'user', $MasterPassword

# Generate a random secure Salt
$SaltBytes = New-Object byte[] 32
[Security.Cryptography.RNGCryptoServiceProvider]::Create().GetBytes($SaltBytes)

# This takes the master password and salt it
# Rfc2898DeriveBytes takes a password, a salt, and an iteration count, and then generates keys through calls to the GetBytes method.
$Rfc2898Deriver = New-Object System.Security.Cryptography.Rfc2898DeriveBytes -ArgumentList $Credentials.GetNetworkCredential().Password, $SaltBytes
$KeyBytes  = $Rfc2898Deriver.GetBytes(32)

# This commands uses Advanced Encryption Standard (AES) algorith
# It will convert the secure string stored in the $SecureKeyString variable to an encrypted standard string using this 256-bit salt key.
# The resulting encrypted standard string is stored in the $StandardString variable.
$EncryptedString = $SecureKeyString | ConvertFrom-SecureString -key $KeyBytes
Write-Host -ForegroundColor Cyan "Encrypted String: " -noNewLine; $EncryptedString

Write-Host -Message "Saving the credentials to configuration file $credentialPath"

$credXML = @{
    api_key = $EncryptedString
    salt = $SaltBytes }
    
if ( (Test-Path $credentialPath) -and ($APIKey) ) {

    $title    = "Credential file already detected in $credentialPath`n"
    $question = "Do you want to replace it?"

    $choices = New-Object Collections.ObjectModel.Collection[Management.Automation.Host.ChoiceDescription]
    $choices.Add((New-Object Management.Automation.Host.ChoiceDescription -ArgumentList '&Yes'))
    $choices.Add((New-Object Management.Automation.Host.ChoiceDescription -ArgumentList '&No'))

    $decision = $Host.UI.PromptForChoice($title, $question, $choices, 1)
    if ($decision -eq 0) {
        Write-Host -ForegroundColor Green "`nConfirmed"
        $credXML | Export-CliXml "$storagePath\shodan.credential" -Force
    } else {
        Write-Host -ForegroundColor Red "`nCancelled, file won't be replaced."
    }
} 

$credXML = Import-Clixml $credentialPath -ErrorAction SilentlyContinue

The below script needs to be able to find the current output cell; this is an easy method to get it.

In [1]:
#Get a master password used to encrypt and decrypt Key
Write-Host -Foreground cyan "`nPlease enter a master password that will be used to encrypt/decrypt this API Key stored locally";
$MasterPassword = Read-Host -AsSecureString

$SaltBytes = $credXML.salt
$Credentials = New-Object System.Management.Automation.PSCredential -ArgumentList 'user', $MasterPassword

$Rfc2898Deriver = New-Object System.Security.Cryptography.Rfc2898DeriveBytes -ArgumentList $Credentials.GetNetworkCredential().Password, $SaltBytes
$KeyBytes  = $Rfc2898Deriver.GetBytes(32)

$SecString = ConvertTo-SecureString $($credXML.api_key) -Key $KeyBytes 
$APIKey = ($SecString | ConvertFrom-SecureString -AsPlainText)

if ($APIKey) {
    Write-Host -ForegroundColor green "`nCredentials have successfully been imported from $credentialPath"
}

The below script needs to be able to find the current output cell; this is an easy method to get it.


Please enter a master password that will be used to encrypt/decrypt this API Key stored locally


 ·········


New-Object: 
Line |
   8 |  … 98Deriver = New-Object System.Security.Cryptography.Rfc2898DeriveByte …
     |                ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | Exception calling ".ctor" with "2" argument(s): "Value cannot be null. (Parameter 'salt')"
InvalidOperation: 
Line |
   9 |  $KeyBytes  = $Rfc2898Deriver.GetBytes(32)
     |  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | You cannot call a method on a null-valued expression.
ConvertTo-SecureString: 
Line |
  11 |  $SecString = ConvertTo-SecureString $($credXML.api_key) -Key $KeyByte …
     |                                      ~~~~~~~~~~~~~~~~~~~
     | Cannot bind argument to parameter 'String' because it is null.
ConvertFrom-SecureString: 
Line |
  12 |  $APIKey = ($SecString | ConvertFrom-SecureString -AsPlainText)
     |                          ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | Cannot bind argument to parameter 'SecureString' because it is null.


In [67]:
if (!(Test-Path -Path "$storagePath")) {
    Write-Verbose -Message 'Seems this is the first time the config has been set.'
    Write-Verbose -Message "Creating folder $storagePath"
    New-Item -ItemType directory -Path $storagePath | Out-Null
}

Write-Verbose -Message "Saving the information to configuration file $storagePath\$APIKey"
"$($EncryptedString)"  | Set-Content  "$storagePath\api.key" -Force
Set-Content -Value $SaltBytes -AsByteStream -Path "$($storagePath)\$saltname" -Force

In [40]:
$SecureKeyString = ConvertTo-SecureString -String $APIKey -AsPlainText -Force
$Credentials = New-Object System.Management.Automation.PSCredential -ArgumentList 'user', $MasterPassword

## Setup Variables

#### Import required modules

#### Enter a domain name or IP address to check
- Example: "jingtoso.com" or "52.1.4.1"

In [281]:
$query = "entergy.com"

#### Logic to determine if its contains letters or not and resolve if it does.

#### Query by Hostname or IP

In [288]:
$hostIP = "4.2.2.1"
$URI = "https://api.shodan.io/shodan/host/$($hostIP)?key=$($APIKey)"

$discovery = Invoke-RestMethod -Method GET -Uri $URI

#### Parse Data

In [289]:
$discovery.data | ForEach -process { 
    Write-Host $_.ip, $_.info, $_.transport, $_.port, $_.product
} 

67240449  udp 53 


In [290]:
$discovery.data | ForEach -process { 
    $_ 
} 


hash      : -103662790
ip        : 67240449
isp       : Level 3 Communications
transport : udp
data      : Version: recursive-main/20717463
            Recursion: enabled
            Resolver name: cns2.lax1.Level3.net
            Resolver ID: pubntp1.lax1.Level3.net
asn       : AS3356
port      : 53
hostnames : {a.resolvers.level3.net}
location  : @{city=; region_code=; area_code=; longitude=-97.822; country_code3=; latitude=37.751; postal_code=; dma_code=; country_code=US; country_name=United States}
dns       : @{software=Version: recursive-main/20717463; recursive=True; resolver_id=pubntp1.lax1.Level3.net; resolver_hostname=cns2.lax1.Level3.net}
timestamp : 6/18/2020 5:10:21 AM
domains   : {level3.net}
org       : Level 3 Communications
os        : 
_shodan   : @{crawler=b0f0de701beef55d9f52a22b2c071d0ea384bc26; ptr=True; id=7781495f-a833-4c70-aaa7-4e34b4df493c; module=dns-udp; options=}
opts      : @{raw=34ef818000010001000000000776657273696f6e0462696e640000100003c00c001000030000